# Walk-through Exercises

### Required imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

### Methods to be used later

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "recap"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#### Let's tackle the _digits dataset_ which is a simple MNIST-like dataset containing 1,797 grayscale 8×8 images representing digits 0 to 9. 
#### We would first need to load the data and split it into test and training set.

In [ ]:
X_digits, y_digits = load_digits(return_X_y=True)

Let's split it into a training set and a test set:

Now let's fit a Logistic Regression model:

Now evaluate the Logistic Regression over the test set. You should get about 0.96 accuracy.

That accuracy will be our baseline to check the benefits of using clustering as a preprocessing step.

Go ahead and **create a pipepline** that will:

We will create a pipeline that will first cluster the training set into 50 

1.   Cluster the training set into 50 clusters. Why 50? Well, we know we have 10 classes so it can be tempting to set $k=10$ but there are many digits that have different handwritting and we need to account for that. Running the K-Means will replace the images with their distances to those $k$ clusters.
2.   Apply Logistic Regression model. Use the following setup:
    * `multi_class="ovr"`
    * `solver="lbfgs"`
    * `max_iter=5000`
    * `random_state=42`
3. Fit the pipeline to `X_train` and `y_train`


Now print the pipeline score. Which accuracy did you get? You should get about 0.97

Now calculate the reduction of the error rate, you can do this by...

$1 - (1 - Pipeline Score) / (1 - Original Score)$

You should get about a 28% reduction of error rate. That means going from 3% error to 2% error.

In [ ]:
1 - (1 - 0.98) / (1 - 0.968888)

Thats a very decent error rate reduction, just by passing a dataset to the classifier which has been clustered before. But we chose the number of clusters $k$ completely arbitrarily, we can surely do better. 

Since K-Means is just a preprocessing step in a classification pipeline, finding a good value for $k$ is much simpler than earlier: there's **no need to perform silhouette analysis** or **minimize the inertia**, the best value of $k$ is simply the one that results in the best classification performance during cross validation.

How could we do that? You are right, by passing the pipeline through `GridSearchCV` (remember to call `GridSearchCV` and fit it to `X_train` and `y_train`). Also remember that we need to look for the number of clusters, so pass the `param_grid` values with a range of clusters from at least 2 to for instance 100.

Hint, refer to the `n_clusters` parameter as `kmeans__n_clusters`. Beware!!!! This will take quite a bit, so run it and go for a coffee. Try it first in not so many clusters, so you can check that your code works. The life of a data scientist allows for many coffee breaks if automatization is done correctly :)

In [ ]:
# Doing 20 here now just to go faster in class
param_grid = dict(kmeans__n_clusters=range(2, 100))



Print the best parameters that we have defined (`kmeans__n_clusters`)

In [ ]:
# We will get bad scores here because we purposedly set the maximum number of clusters to 20


Print the resulting score of the GridSearchCV, you should get about 98 to 99% accuracy. And you should get that you got your best values with $k=99$

In [ ]:
# With 99 clusters we will get about 99%, but we only used  k=19


### Clustering for Semi-supervised Learning

Another use case for clustering is in semi-supervised learning, when we have plenty of unlabeled instances and very few labeled instances.

Let's look at the performance of a logistic regression model when we only have 50 labeled instances. We can reduced the MNIST dataset loaded previously to just the last 50 instances.

In [ ]:
n_labeled = 50

It's much less than earlier of course. Let's see how we can do better. First, let's cluster the training set into 50 clusters, then for each cluster let's find the image closest to the centroid. We will call these images the **representative** images:

In [ ]:
k = 50

Now let's plot these representative images and label them manually:

In [ ]:
"""
# You will need to label here whatever appeared 
in your most representative images according to your random_state
"""
y_representative_digits = np.array([
    2, 6, 0, 3, 4, 2, 8, 1, 7, 9,
    5, 9, 8, 3, 6, 1, 8, 7, 2, 0,
    4, 2, 9, 9, 5, 6, 7, 1, 6, 9,
    4, 3, 1, 0, 8, 4, 7, 2, 5, 4,
    5, 2, 2, 8, 5, 9, 2, 7, 7, 7])

In [ ]:
# Above in the last row in position 5 is not an "8" is an "1"
#y_train[representative_digit_idx]

Now we have a dataset of the most representative instances labeled. Unfortunately we only have 50 labeled instances, but instead of being completely random instances, each of them is a representative image of its cluster. Let's see if the performance is any better:

Wow! We jumped from accuracy although we are still only training the model on 50 instances. Since it's often costly and painful to label instances, especially when it has to be done manually by experts, it's a good idea to make them label representative instances rather than just random instances.

So this is a very good technique to first take those most representative instances and label them, instead of labelling randomly.

But perhaps we can go one step further: what if we propagated the labels to all the other instances in the same cluster?

In [ ]:
# Create an empty matrix with the length of X_train
y_train_propagated = np.empty(len(X_train),
                              dtype=np.int32)

"""
Propagate the cluster instances into that matrix by propagating the representative
digit value to the full cluster that has that value
"""
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

We got a tiny little accuracy boost. Better than nothing, but we should probably have propagated the labels only to the instances closest to the centroid, because by propagating to the full cluster, we have certainly included some outliers. This might lead to a lot of problems, such as overfitting. Let's only propagate the labels to the 30th percentile closest to the centroid:

In [ ]:
percentile_closest = 30

# Create an empty matrix with the length of X_train
X_cluster_dist = X_digits_dist[np.arange(len(X_train)),
                               kmeans.labels_]

for i in range(k):
    # Get the clusters that has the i value
    in_cluster = (kmeans.labels_ == i)

    # Index the newly created X_cluster_dist with in_cluster
    cluster_dist = X_cluster_dist[in_cluster]

    # Set up the percentile distance by using np.percentile
    cutoff_distance = np.percentile(cluster_dist,
                                    percentile_closest)

    # Set a value to be above the cluster distance
    above_cutoff = (X_cluster_dist > cutoff_distance)

    # Set X_cluster_dist values to -1 when those are above cutoff distance
    X_cluster_dist[in_cluster & above_cutoff] = -1

In [ ]:
# Get the real propagated values (not -1)
partially_propagated = (X_cluster_dist != -1)

# Index those values to X_train
X_train_partially_propagated = X_train[partially_propagated]

# Index those values to y_train
y_train_partially_propagated = y_train_propagated[partially_propagated]

Nice! With just 50 labeled instances (just 5 examples per class on average!), we got this performance, which is pretty close to the performance of logistic regression on the fully labeled _digits_ dataset. We got a bit lower performance but results should be more stable.

This is because the propagated labels are actually pretty good: their accuracy is very close to 99%:

In [ ]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])